In [22]:
suppressMessages(library(rwwa))
png_res <- 240

In [2]:
# mdl_args <- list("hdwi_chile2026" = list(dist = "gev", type = "shiftscale", lower = F, covnm = c("gmst")),
#                  "hdwi_patagonia" = list(dist = "gev", type = "shift", lower = F, covnm = c("gmst")),
#                  "pr-ndj_chile2026" = list(dist = "norm_logt", type = "shift", lower = T, covnm = c("gmst", "samdet", "nino34")),
#                  "pr-ndj_patagonia" = list(dist = "norm_logt", type = "shift", lower = T, covnm = c("gmst", "samdet", "nino34")),
#                  "pet-ndj_chile2026" = list(dist = "norm", type = "shift", lower = F, covnm = c("gmst", "samdet", "nino34")),
#                  "pet-ndj_patagonia" = list(dist = "norm", type = "shift", lower = F, covnm = c("gmst", "samdet", "nino34")),
#                  "epr-ndj_chile2026" = list(dist = "norm", type = "shift", lower = T, covnm = c("gmst", "samdet", "nino34")),
#                  "epr-ndj_patagonia" = list(dist = "norm", type = "shift", lower = T, covnm = c("gmst", "samdet", "nino34")))


# cov_df <- merge(merge(read.table("ts-obs/gmst-smoothed.dat", col.names = c("year", "gmst")),
#                       load_ts("ts-obs/sam-fires_nino34det-ndj.dat", col.names = c("year", "nino34"))),
#                 load_ts("ts-obs/sam-fires_sam-dec.dat", col.names = c("year", "sam")))
# cov_df$gmst <- cov_df$gmst - cov_df$gmst[cov_df$year == 2026]
# cov_df$samdet <- resid(lm(sam ~ gmst, cov_df))


# cov_2026 <- cov_df[cov_df$year == 2026,c("gmst", "sam", "samdet", "nino34"),drop = F]
# cov_cf <- rbind("pi" = cov_2026 - c(1.3, 0, 0, 0),
#                 "samneutral" = c(cov_2026$gmst, 0, 0, cov_2026$nino34),
#                 "ensoneutral" = c(cov_2026$gmst, cov_2026$sam, cov_2026$samdet, 0),
#                 "bothneutral" = c(cov_2026$gmst, 0, 0, 0))

# cov_list <- list("gmst-only" = "gmst", "gmst+nino" = c("gmst", "nino34"), "gmst+sam" = c("gmst", "sam"), 
#                  "gmst+nino+sam" = c("gmst", "sam", "nino34")
#                  ,"gmst+samdet" = c("gmst", "samdet"),"gmst+nino+samdet" = c("gmst", "samdet", "nino34")
#                 )

# Tables of results

In [18]:
varnm <- "rx1day-ondjfm"
rnm <- "n"

res <- read.csv(paste0("res-obs_",varnm,"_",rnm,".csv"), row.names = "X")
res <- res[!grepl("naodet", rownames(res)),]

In [19]:
cols <- c("event", "dI.rel.pi", "dI.rel.naoneutral", "return.period", "PR.pi", "PR.naoneutral", "aic")
sapply(cols, function(cnm) {
    apply(signif(res[,grepl(cnm, colnames(res)),drop = F], 3), 1, function(r) {
        paste0(r[1], " (",r[2], ", ", r[3], ")")
    })
})

,event,dI.rel.pi,dI.rel.naoneutral,return.period,PR.pi,PR.naoneutral,aic
chirps_gmst-only,"61.2 (61.2, 61.2)","2.24 (-31.7, 36.7)","0 (0, 0)","1.9 (1.28, 5)","1.07 (0.261, 3.09)","1 (1, 1)","386 (359, 399)"
chirps_gmst+nao,"61.2 (61.2, 61.2)","2.16 (-32, 42.4)","-0.194 (-6.65, 8.98)","1.91 (1.24, 5.94)","1.07 (0.224, 3.27)","0.994 (0.661, 1.29)","388 (360, 400)"
cpc_gmst-only,"34.3 (34.3, 34.3)","4.4 (-22.7, 34.4)","0 (0, 0)","1.53 (1.16, 3.14)","1.12 (0.443, 2.11)","1 (1, 1)","352 (326, 365)"
cpc_gmst+nao,"34.3 (34.3, 34.3)","5.55 (-18, 38.6)","7.41 (2.08, 13.9)","1.33 (1.07, 2.08)","1.13 (0.573, 2.14)","1.18 (1.03, 1.49)","347 (320, 361)"
eobs_gmst-only,"45.4 (45.4, 45.4)","35.4 (12.4, 61.6)","0 (0, 0)","2.21 (1.43, 4.78)","6.56 (2.24, 39.6)","1 (1, 1)","548 (515, 567)"
eobs_gmst+nao,"45.4 (45.4, 45.4)","39.3 (16.4, 70.5)","2.66 (-1.04, 6.67)","1.96 (1.29, 4.05)","7.12 (2.71, 46.9)","1.1 (0.958, 1.35)","549 (515, 568)"
era5_gmst-only,"52 (52, 52)","17 (-1.68, 40.5)","0 (0, 0)","3.58 (1.81, 10.4)","3.18 (0.885, 407)","1 (1, 1)","567 (539, 586)"
era5_gmst+nao,"52 (52, 52)","19.1 (-1.01, 47.3)","1.61 (-1.75, 5.29)","3.22 (1.56, 9.35)","3.45 (0.925, Inf)","1.09 (0.888, 1.36)","569 (539, 586)"
mswep_gmst-only,"69.2 (69.2, 69.2)","29.4 (-9.9, 85.8)","0 (0, 0)","4.91 (1.63, 30.8)","12 (0.396, Inf)","1 (1, 1)","388 (367, 401)"
mswep_gmst+nao,"69.2 (69.2, 69.2)","29.4 (-10.3, 86.6)","1.61 (-3.52, 9.09)","4.61 (1.63, 23)","9.46 (0.46, Inf)","1.1 (0.805, 1.82)","390 (367, 402)"


# Figures for obs discussion

In [43]:
df <- merge(cov_df, load_ts(paste0("ts-obs/sam-fires_",varnm,"_",rnm,"_era5land.dat"), col.names = c("year", "x")))
if (varnm %in% c("pr-ndj")) df$x <- df$x * 92

In [44]:
mdl <- do.call(fit_ns, append(mdl_args[[paste0(varnm,"_",rnm)]], list("data" = df, "varnm" = "x")))

## Trends vs covariates

In [45]:
png(paste0("fig/covtrend_",varnm,"_",rnm,".png"), h = png_res, w = png_res * 3); {
    prep_window(c(1,3), cex.axis = 1.25, mar = c(4,2,3,1), oma = c(0,3,0,0))

    plot_covtrend(mdl, "gmst", ylab = "", xlab = "", legend_pos = NA)
    mtext("NDJ precipitation (mm)", side = 2, line = 3.25)
    mtext("GMST anomaly wrt 2026", side = 1, line = 2.75)
    mtext("(a)", side = 3, line = 0.75, adj = 0, font = 2)
    
    plot_covtrend(mdl, "nino34", ylab = "", xlab = "", legend_pos = NA)
    mtext("Detrended Nino3.4 index", side = 1, line = 2.75)
    mtext("(b)", side = 3, line = 0.75, adj = 0, font = 2)
    
    plot_covtrend(mdl, "samdet", ylab = "", xlab = "", legend_pos = NA)
    mtext("Detrended SAM index", side = 1, line = 2.75)
    mtext("(c)", side = 3, line = 0.75, adj = 0, font = 2)
}; dev.off()

agg_record_300909875 
                   2

## Trends & GMST return level plot

In [46]:
png(paste0("fig/trend-and-rl_",varnm,"_",rnm,".png"), h = png_res*1.5, w = png_res*1.5 * 3); {
    
    prep_window(c(1,2), cex.axis = 1.25, cex.lab = 1.25, mar = c(4,2,2.5,1), oma = c(0,3,0,0))

    plot_trend(mdl, ylab = "", xlab = "", legend_pos = NA)
    ylab_text = switch(varnm,
                       "pr-ndj" = "NDJ precipitation (mm)",
                       "hdwi" = "HDWI (kPa m/s)",
                       "x")
    mtext(ylab_text, side = 2, line = 3.25, cex = 1.25)
    
    mtext("Year", side = 1, line = 2.75, cex = 1.5)
    mtext("(a)", side = 3, line = 0.75, adj = 0, font = 2)

    plot_returnlevels(mdl, cov_f = cov_2026, cov_cf = cov_cf["pi",,drop = F], ylab = "", xlab = "",
                      model_desc = F, legend_labels = c("2026", "1.3°C cooler climate"), legend_pos = "bottomright")
    mtext("Return period (years)", side = 1, line = 2.75, cex = 1.5)
    mtext("(b)", side = 3, line = 0.75, adj = 0, font = 2)

}; dev.off()

agg_record_665579472 
                   2

## Return level plots

In [ ]:
# png(paste0("fig/rlplots_",varnm,"_",rnm,".png"), h = png_res, w = png_res * 3); {
#     prep_window(c(1,3), cex.axis = 1.25, mar = c(4,2,3,1), oma = c(0,3,0,0))

#     plot_returnlevels(mdl, cov_f = cov_2026, cov_cf = cov_cf["pi",,drop = F], ylab = "", legend_pos = NA, nsamp = 5)
#     mtext("NDJ precipitation (mm)", side = 2, line = 3.25)
#     mtext("(a)", side = 3, line = 0.75, adj = 0, font = 2)

#     plot_returnlevels(mdl, cov_f = cov_2026, cov_cf = cov_cf["ensoneutral",,drop = F], ylab = "", legend_pos = NA, nsamp = 5)
#     mtext("(b)", side = 3, line = 0.75, adj = 0, font = 2)
    
#     plot_returnlevels(mdl, cov_f = cov_2026, cov_cf = cov_cf["samneutral",,drop = F], ylab = "", legend_pos = NA, nsamp = 5)
#     mtext("(c)", side = 3, line = 0.75, adj = 0, font = 2)
    
# }; dev.off()

# Model evaluation tables

In [139]:
varnm <- "pr-ndj"
rnm <- "patagonia"

obs_res <- read.csv(paste0("res-obs_",varnm,"_",rnm,".csv"), row.names = "X")[c("gmst-only"),,drop = F]
mdl_res <- read.csv(paste0("res-cordex_",varnm,"_gmst-only_",rnm,"_with-eval.csv"), row.names = "X.1")

In [140]:
obs_df <- data.frame("Event" = signif(obs_res$event.magnitude_est,3),
                     "Seasonal" = "",
                     "Spatial" = "", 
                     "Scale parameter" = apply(signif(obs_res[,grepl("sigma0", colnames(obs_res)),drop = F], 3), 1, function(r) paste0(r[1], " (",r[2], ", ", r[3], ")")),
                     "Shape parameter" = apply(signif(obs_res[,grepl("shape", colnames(obs_res)),drop = F], 3), 1, function(r) paste0(r[1], " (",r[2], ", ", r[3], ")")),
                     "Overall" = "",
                     "Keep" = "")
rownames(obs_df) <- c("ERA5-Land")

mdl_df <- data.frame("Event" = signif(mdl_res$rp_value,3),
                     "Seasonal" = mdl_res$seasonal,
                     "Spatial" = mdl_res$spatial, 
                     "Scale parameter" = apply(signif(mdl_res[,grepl("sigma0", colnames(mdl_res)),drop = F], 3), 1, function(r) paste0(r[1], " (",r[2], ", ", r[3], ")")),
                     "Shape parameter" = apply(signif(mdl_res[,grepl("shape", colnames(mdl_res)),drop = F], 3), 1, function(r) paste0(r[1], " (",r[2], ", ", r[3], ")")),
                     "Overall" = mdl_res$cat,
                     "Keep" = mdl_res$include)
rownames(mdl_df) <- gsub("i1p1_", "i1p1 ",rownames(mdl_df))

spacer_df <- data.frame(sapply(colnames(obs_df), function(cnm) rep("",2)), row.names = c("", "CORDEX"))

# Model results

In [258]:
varnm <- "pr-ndj"
rnm <- "chile2026"

obs_res <- read.csv(paste0("res-obs_",varnm,"_",rnm,".csv"), row.names = "X")[c("gmst-only"),,drop = F]

mdl_res <- read.csv(paste0("res-cordex_",varnm,"_gmst-only_",rnm,"_with-eval.csv"), row.names = "X.1")
print(nrow(mdl_res))
mdl_res <- mdl_res[mdl_res$include == "Y",]
print(nrow(mdl_res))

[1] 18
[1] 8


In [250]:
cols <- switch(varnm, "hdwi" = c("dI.abs", "PR"), "pr-ndj" = c("dI.rel", "PR"))

mdl_df <- data.frame(cbind(sapply(cols, function(cnm) apply(signif(mdl_res[,grepl(paste0("attr_",cnm), colnames(mdl_res)),drop = F], 3), 1, 
                                                           function(r) paste0(r[1], " (",r[2], ", ", r[3], ")")))),
                     cbind(sapply(cols, function(cnm) apply(signif(mdl_res[,grepl(paste0("proj_",cnm), colnames(mdl_res)),drop = F], 3), 1, 
                                                           function(r) paste0(r[1], " (",r[2], ", ", r[3], ")")))))
rownames(mdl_df) <- gsub("i1p1_", "i1p1 ",rownames(mdl_df))


In [244]:
obs_df <- data.frame(t(sapply(cols, function(cnm) apply(signif(obs_res[,grepl(cnm, colnames(obs_res)),drop = F], 3), 1, 
                                                        function(r) paste0(r[1], " (",r[2], ", ", r[3], ")")))))
colnames(obs_df) <- colnames(mdl_df)[1:length(colnames(obs_df))]
rownames(obs_df) <- c("ERA5-Land")


In [245]:
df <- data.frame(rbind.fill(obs_df, mdl_df), row.names = c(rownames(obs_df), rownames(mdl_df)))

In [246]:
cat(rnm,varnm)
df[,c(1,3,2,4)]

patagonia pr-ndj

,dI.rel,dI.rel.1,PR,PR.1
,<chr>,<chr>,<chr>,<chr>
ERA5-Land,"-35 (-58.5, -3.46)",NA,"9.08 (1.22, 104)",NA
SAM-22_HadGEM2-ES_r1i1p1 RegCM4-7,"-15.3 (-31.1, 3.01)","-8.8 (-16.2, -1.46)","2.6 (0.863, 17.3)","1.45 (1.07, 1.93)"
SAM-22_HadGEM2-ES_r1i1p1 REMO2015,"-14.6 (-30, 3.18)","-11.5 (-19, -4.25)","2.49 (0.853, 14.6)","1.64 (1.22, 2.17)"
SAM-22_NorESM1-M_r1i1p1 RegCM4-7,"-6.36 (-27.3, 17.6)","-15.4 (-26, -5.09)","1.53 (0.44, 18.1)","1.92 (1.29, 2.68)"
SAM-22_NorESM1-M_r1i1p1 REMO2015,"-8.62 (-31.3, 16.6)","-16 (-27, -5.73)","1.79 (0.449, 20.4)","1.96 (1.34, 2.75)"
SAM-44_HadGEM2-ES_r1i1p1 RegCM4-3,"-8.72 (-28, 14.6)","-3.47 (-11.6, 3.44)","1.56 (0.576, 8.32)","1.17 (0.841, 1.64)"
